<div align="center">

# 
# 🎬 ActionMesh
### Animated 3D Mesh Generation with Temporal 3D Diffusion

**[Meta Reality Labs](https://ai.facebook.com/research/)** &nbsp;•&nbsp; **[SpAItial](https://www.spaitial.ai/)** &nbsp;•&nbsp; **[University College London](https://geometry.cs.ucl.ac.uk/)**

[Remy Sabathier](https://remysabathier.github.io/RemySabathier/), [David Novotny](https://d-novotny.github.io/), [Niloy J. Mitra](http://www0.cs.ucl.ac.uk/staff/n.mitra/), [Tom Monnier](https://tmonnier.com/)

[![Paper](https://img.shields.io/badge/Paper-ActionMesh-red)](https://arxiv.org/abs/2601.16148)
[![Project Page](https://img.shields.io/badge/Project_Page-green)](https://remysabathier.github.io/actionmesh/)
[![GitHub](https://img.shields.io/badge/GitHub-black)](https://github.com/facebookresearch/actionmesh)
[![HuggingFace](https://img.shields.io/badge/%F0%9F%A4%97%20HuggingFace-Demo-blue)](https://huggingface.co/spaces/facebook/ActionMesh)

<img src="https://github.com/facebookresearch/actionmesh/raw/main/assets/docs/teaser.jpg" width="700">

</div>

---

**ActionMesh** transforms a video into an animated 3D mesh with consistent topology across all frames.

| GPU | Inference Time |
|-----|----------------|
| Colab T4 | ~15 min |
| A100 | ~90 sec |
| H100 | ~45 sec |

⚡ **Requirements**: GPU runtime required (T4 minimum, A100 recommended for faster inference).

## 📦 Setup

### 1. Install ActionMesh

In [ ]:
import numpy as np

# Check if numpy needs downgrade (Colab has numpy>=2 by default)
if int(np.__version__.split('.')[0]) >= 2:
    print("⚠️ Downgrading numpy to <2 (required by ActionMesh)...")
    !pip install -q "numpy<2"
    print("🔄 Please restart runtime: Runtime > Restart session, then re-run this cell")
else:
    import os
    if not os.path.exists("/content/actionmesh"):
        !git config --global url."https://github.com/".insteadOf "git@github.com:"
        !git clone https://github.com/facebookresearch/actionmesh.git /content/actionmesh
        %cd /content/actionmesh
        !git submodule update --init --recursive
        !pip install -q -r requirements.txt
        !pip install -q -e .
    else:
        %cd /content/actionmesh
        print("✅ ActionMesh already installed!")
    print("✅ Setup complete!")

### 2. Install Blender 3.5.1

> **Note**: Blender is only required to export animated `.glb` files.

In [ ]:
# @title Install Blender 3.5.1 { display-mode: "form" }
# @markdown Required for generating animated GLB files.

import os

BLENDER_PATH = "/content/blender-3.5.1-linux-x64/blender"

if not os.path.exists(BLENDER_PATH):
    print("📥 Downloading Blender 3.5.1...")
    !wget -q --show-progress -P /content https://download.blender.org/release/Blender3.5/blender-3.5.1-linux-x64.tar.xz
    print("📦 Extracting...")
    !tar -xf /content/blender-3.5.1-linux-x64.tar.xz -C /content
    !rm /content/blender-3.5.1-linux-x64.tar.xz
    print("✅ Blender installed!")
else:
    print("✅ Blender already installed!")

# Verify installation
!{BLENDER_PATH} --version | head -1

---

## 🚀 Run ActionMesh

**Supported inputs:**
- `.mp4` video file
- Folder containing PNG images
- Number of frames: **16 to 31** (additional frames are ignored)

> 💡 **Tip for custom videos:** For best results, the subject should be isolated on a flat background. We recommend using the [SAM2 demo](https://sam2.metademolab.com/demo) to segment your subject. See our [SAM2 extraction guide](https://github.com/facebookresearch/actionmesh/blob/main/assets/docs/sam2_extraction_guide.md) for detailed instructions.

In [ ]:
# @title Run ActionMesh

# Change the input path to use your own video.
input_path = "assets/examples/davis_camel"

!python -u inference/video_to_animated_mesh.py \
    --input "{input_path}" \
    --fast \
    --low_ram \
    --dtype float16 \
    --seed 44 \
    --blender_path "{BLENDER_PATH}"

# Note: On A100/H100 GPUs, we recommend --dtype bfloat16

---

## 🎬 Visualize Results

### Interactive 3D Viewer

In [ ]:
# @title Setup 3D Viewer { display-mode: "form" }
# @markdown Run this cell to enable the interactive 3D model viewer.

!pip -q install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

import base64
from IPython.display import HTML, display
from pathlib import Path

def show_glb(glb_path, width=800, height=500):
    """Display an animated GLB model in an interactive 3D viewer."""
    if not Path(glb_path).exists():
        print(f"❌ File not found: {glb_path}")
        return

    b64 = base64.b64encode(open(glb_path, "rb").read()).decode("utf-8")

    display(HTML(f"""
    <style>
        .viewer-container {{
            border-radius: 12px;
            overflow: hidden;
            box-shadow: 0 4px 20px rgba(0,0,0,0.3);
        }}
    </style>
    <script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.min.js"></script>
    <div class="viewer-container">
        <model-viewer
            src="data:model/gltf-binary;base64,{b64}"
            style="width:{width}px; height:{height}px; background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);"
            camera-controls
            shadow-intensity="0"
            exposure="1"
            neutral-lighting
            autoplay
            camera-orbit="140deg 80deg 3.5m"
            field-of-view="30deg"
            touch-action="pan-y"
            ar
            ar-modes="webxr scene-viewer quick-look">
        </model-viewer>
    </div>
    """))

print("✅ 3D Viewer ready!")

In [ ]:
# @title Display Result { display-mode: "form" }
# @markdown View the generated animated mesh.

from pathlib import Path

# Auto-detect output path from input
input_name = Path(input_path).stem
output_dir = f"/content/actionmesh/outputs/{input_name}"
output_glb = f"{output_dir}/animated_mesh.glb"

show_glb(output_glb)

---

<div align="center">

## 📚 Citation

If you find ActionMesh useful, please cite our paper:

```bibtex
@inproceedings{ActionMesh2025,
  author = {Remy Sabathier, David Novotny, Niloy J. Mitra, Tom Monnier},
  title = {ActionMesh: Animated 3D Mesh Generation with Temporal 3D Diffusion},
  year = {2025},
}
```

---

</div>